In [1]:
from jupyter_dash import JupyterDash
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import base64

##importing needed basics for authentication
import dash_auth
from users import USERNAME_PASSWORD_PAIRS


###########################
# Data Manipulation / Model
###########################

# class read method must support return of cursor object and accept projection json input
clients = pd.read_csv("DAT220.csv")
df = pd.read_csv("DAT220.csv")
#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Bubba Gump Example')

##creating authentication step to access API
##hiding stored user values to protect username/pw
auth = dash_auth.BasicAuth(
    app,
    USERNAME_PASSWORD_PAIRS
)


#placing custom image on API
image_filename = 'image.jpg' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('BubbaGump Data Dashboard'))),
    html.Hr(),
    html.Div(id="query-out"),
    html.Center(html.H1('Byron Laferriere Productions')),
    html.Center(html.H2('Check out link below to see more work!')),
    html.Div([
    html.A([
            html.Center(html.H3('https://byronlaferriere.github.io/'))],
                        href='https://byronlaferriere.github.io/'),
         ]),
        # logo with URL anchor
    html.Div([
        html.A([
            html.Center(html.Img(src='data:image/jpg;base64,{}'.format(encoded_image.decode())))],
                        href='https://www.facebook.com/bubbagumprestaurants/'),
        ]),
    html.Hr(),
    html.Div(
         
# code for the interactive filtering options using radio buttons
        dcc.RadioItems(id='radio-id',
            options=[
                {'label': 'City', 'value': 'city'},
                {'label': 'County', 'value': 'county'},
                {'label': 'State', 'value': 'state'},
                {'label': 'Zip', 'value': 'zip'},
                {'label': 'Zip 2', 'value': 'ZIP_2'},
                {'label': 'Restaurant', 'value': 'Restaurant'},
                {'label': 'Restaurant Visits', 'value': 'RES_VISITS'},
                {'label': 'Web Purchases?', 'value': 'WEB_PURCH_YN'},
                {'label': 'Online Spending', 'value': 'Webstore_Spend'},
                {'label': 'Website Visits', 'value': 'WEB_VISITS'},
                {'label': 'Third-Party Vendor Spending', 'value': 'THIRD_SPEND'},
                {'label': 'Third-Party Vendor Visits', 'value': 'THIRD_VISITS'},
                {'label': 'Age', 'value': 'Age'},
                {'label': 'Married?', 'value': 'Married_YN'},
                {'label': 'Married(Binary)', 'value': 'MARR_BIN'},
                {'label': 'Income', 'value': 'Income'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value = 'city',
            labelStyle={'display': 'inline-block'}
        ) 
    ),

    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
        # features for interactive data table to make it user-friendly for client
        style_table = {'overflowX': 'auto'},
        style_cell = {'minWidth': '180px', 'width': '180px', 'maxWidth': '180px',
        'whiteSpace': 'normal'},
        editable = False,
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
        style={'display' : 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('radio-id', 'value')])

def update_dashboard(input_value):
    df = pd.read_csv("DAT220.csv")
    clients = pd.read_csv("DAT220.csv")
    # query logic for interactive data filtering (radio buttons)
    if input_value == 'City':
        df = pd.DataFrame.from_records(clients.city({
            
        }))
    elif input_value == 'State':
        df = pd.DataFrame.from_records(clients.state({
            
        }))
    elif input_value == 'Income':
        df = pd.DataFrame.from_records(clients.Income({
           
        }))
    elif input_value == 'reset':
        df = pd.DataFrame.from_records(clients({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
    
        
    return (data,columns)


# update styles
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# pie chart with the percentage of client base
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    df_grouped = dff[['state', '1']].groupby(['city']).agg('count').reset_index()
    df_grouped = df_grouped.rename(columns = {'1': 'Count'})
    
    fig = px.pie(df_grouped, values='Count', names='state', title="City and State")
    return [
        dcc.Graph(figure=fig)
    ]

# geolocation map 
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
# code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # return all markers from page being viewed
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[61.217381,-149.863129], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")] +
            # Marker with tool tip and popup
            [dl.Marker(position=[row['location_lat'],row['location_long']], children=[
                dl.Tooltip(row['state']),
                dl.Popup([
                     html.H1("Client Name"),
                     html.P(row['first_name, last_name'])
                ])
            ]) for index, row in dff.iterrows()]
        )
    ]

app




C:\Users\byron\anaconda3\lib\site-packages\dash_auth\plotly_auth.py:16: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\byron\anaconda3\lib\site-packages\dash_auth\plotly_auth.py:17: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
app.run_server(debug=True)

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

In [15]:
df

,first_name,last_name,city,county,state,zip,ZIP_2,Restaurant,RES_VISITS,WEB_PURCH_YN,Webstore_Spend,WEB_VISITS,THIRD_SPEND,THIRD_VISITS,Age,Married_YN,MARR_BIN,Income
0,Lenna,Paprocki,Anchorage,Anchorage,AK,99501,99,155,2,1,191,3,63,1,28,N,0,42
1,Roxane,Campain,Fairbanks,Fairbanks North Star,AK,99708,99,60,1,0,0,1,0,0,49,Y,1,112
2,Erick,Ferencz,Fairbanks,Fairbanks North Star,AK,99712,99,155,2,0,0,0,0,0,44,Y,1,74
3,Penney,Weight,Anchorage,Anchorage,AK,99515,99,90,1,0,0,0,26,1,19,N,0,47
4,Wilda,Giguere,Anchorage,Anchorage,AK,99501,99,131,3,1,223,2,0,0,25,Y,1,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Cecilia,Colaizzo,Madison,Dane,WI,53717,53,46,1,0,0,0,0,0,73,N,0,67
496,Janey,Gabisi,Madison,Dane,WI,53715,53,131,2,0,0,1,165,2,28,Y,1,37
497,Natalie,Fern,Rock Springs,Sweetwater,WY,82901,82,34,1,0,0,0,0,0,22,Y,1,58
498,Lauran,Burnard,Riverton,Fremont,WY,82501,82,21,1,0,0,0,106,2,29,Y,1,53
